In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sc
                
# 创建图表
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))

# 数据准备
labels = np.array(['MI', 'VIF', 'SSIM', 'Qabf', 'Qc', 'Qp', 'Qcv'])
num_vars = len(labels)

# 角度计算
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]  # 闭合


# 需要绘制的数据值
values_ours_ = np.array([2.54, 0.8644, 1.4423, 0.6971, 0.9197, 0.9406, 187.6])
values_ours = np.array([2.34, 0.8144, 1.4023, 0.5971, 0.8301, 0.9406, 1/200.6])
values_TextIF = np.array([2.0902, 0.7466, 1.3274, 0.5256, 0.8157, 0.9293, 1/354.5596])
values_CDDFuse = np.array([2.21, 0.7482, 1.3517, 0.5458, 0.8178, 0.9327, 1/258.91])
values_U2Fusion = np.array([2.03, 0.7681, 1.335, 0.5017, 0.8127, 0.9256, 1/374.7083])

# normalize it
values_all = np.stack([values_ours, values_TextIF, values_CDDFuse, values_U2Fusion], axis=0)
values_all = values_all / values_all.max(axis=0, keepdims=True)
colors = ['blue', 'red', 'green', 'orange']
temps_for_each_metric = np.array([0.2, 0.3, 0.2, 0.2, 0.05, 0.02, 0.6])

def softmax_with_temp(values, temp=0.6):
    return sc.softmax(values / temp, axis=0)

values_all = softmax_with_temp(values_all, temps_for_each_metric)


def plot_radar(values, ax, fill=False, color='blue'):
    print(values)

    # 闭合数据环，起点和终点相同
    values = np.concatenate((values, [values[0]]))

    # 绘制雷达图
    if fill:
        ax.fill(angles, values, color=color, alpha=0.25)
    lin = ax.plot(angles, values, color=color, linewidth=2)

    # add dot of each datapoint
    ax.scatter(angles, values, color=color, s=60)

    return values, lin

lins = []
values_alls = []
for i in range(values_all.shape[0]):
    values, lin = plot_radar(values_all[i], ax, fill=False if i != 0 else True, color=colors[i])
    values_alls.append(values)
    lins.append(lin)
    
values_ours = values_alls[0]

# add text
for i, label in enumerate(labels):
    ax.text(angles[i]+0.1, values_ours[i] + 0.05, str(round(values_ours_[i], 3)), ha='center', va='center', fontsize=14)

# 添加标签
ax.set_yticklabels([])
ax.set_xticks(angles[:-1])  # 去掉最后一个角度，不闭合标签
ax.set_ylim(0, 0.45)
# 斜体
ax.set_xticklabels(labels, fontsize=16, fontstyle='italic')  # 标签和角度数量一致

# add legend with line and shift to upper right
# add some shift 
ax.legend([l[0] for l in lins], ['Ours', 'CDDFuse', 'TextIF', 'U2Fusion'], loc='upper right', fontsize=14, frameon=True, bbox_to_anchor=(1.25, 1.1))

plt.show()
